In [1]:
# %%capture
# Install the required libraries
%pip install --upgrade pip
%pip install uv
!uv pip install --system --no-progress --link-mode=symlink unsloth vllm sentencepiece protobuf datasets wandb huggingface_hub kagglehub[hf-datasets] hf_transfer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 119.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Using Python 3.11.11 environment at: /usr
Resolved 187 packages in 1.02s
 Downloaded tokenizers
 Downloaded torchaudio
 Downloaded uvloop
 Downloaded xgrammar
 Downloaded mistral-common
 Downloaded pycountry
 Downloaded torchvision
 Downloaded numba
 Downloaded llguidance
 Downloaded llvmlite
 Downloaded hf-xet
 Downloaded xformers
 Downloaded bitsandbytes
 Downloaded nvidia-cusparselt-cu12
 Downloaded triton
 Downloaded vllm
 Downloaded ray
 Downloaded torch
Prepared 59 packages in 21.74s
Uninstalled 21 packages in 2.60s
Installed 67 packages in 1.71s
 + airpor

## Load and Explore the Dataset

First, let's load the dataset and explore its structure.

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from datasets import Dataset

# Set the path to the file you'd like to load
dataset_file_path = "wiki.csv"

# Load the latest version
dataset:Dataset = kagglehub.dataset_load(
  KaggleDatasetAdapter.HUGGING_FACE,
  "saurabhshahane/bangla-wikipedia",
  dataset_file_path,
  # Provide any additional arguments like 
  # sql_query, hf_kwargs, or pandas_kwargs. See 
  # the documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterhugging_face
)

dataset = dataset.select(range(1000))

print("Dataset:", dataset)

Dataset: Dataset({
    features: ['id', 'text', 'title', 'url'],
    num_rows: 1000
})


In [3]:
print("Dataset Type:", type(dataset))
print("Dataset Columns:", dataset.column_names)
print("Dataset Shape:", dataset.shape)
print("Dataset Features:", dataset.features)
print("Dataset Example:", dataset[0])

Dataset Type: <class 'datasets.arrow_dataset.Dataset'>
Dataset Columns: ['id', 'text', 'title', 'url']
Dataset Shape: (1000, 4)
Dataset Features: {'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None)}
Dataset Example: {'id': 2341, 'text': 'বংশী বাংলাদেশের একটি ক্ষুদ্রতম উপজাতি। এরা টাঙ্গাইল জেলার "মহানান্দপুর" এবং "দন্দোনিয়া" নামে পাশাপাশি দুইটি গ্রামের বসবাস করে। তারা নিজেদেরকে "সূর্য-বংশী" বলে থাকে।\n', 'title': 'বংশী', 'url': 'https://bn.wikipedia.org/wiki?curid=2341'}


## Load Sarvam-1 Model and Tokenizer

We'll load the pretrained Sarvam-1 model and tokenizer for fine-tuning.

In [4]:
# First empty the CUDA memory

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    with torch.no_grad():
        torch.cuda.empty_cache()

In [5]:
# @title Loading Sarvam-1 from Huggingface
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "sarvamai/sarvam-1",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    # token = "hf_...", # use one if using gated models
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-04-30 09:45:38.499719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746006338.780521      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746006338.857475      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-30 09:46:00 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 04-30 09:46:00 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.3: Fast Llama patching. Transformers: 4.51.1. vLLM: 0.8.5.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/775k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

sarvamai/sarvam-1 does not have a padding token! Will use pad_token = <unk>.


### Apply Parameter-Efficient Fine-Tuning (PEFT) to the pre-trained Gemma-3-4b model (unsloth provided)

In [6]:
model = FastModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model.embed_tokens` require gradients


In [7]:
# @title Prompt and Dataset Loading (upload the poems_dataset.csv) to Google colab

from datasets import load_dataset
import pandas as pd

prompt = """
### Instruction:
{}
### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

# Formatting function
def formatting_prompts_func(examples):
    instructions = examples["title"]
    responses = examples["text"]
    texts = []
    for instruction, response in zip(instructions, responses):
        text = prompt.format(instruction, response) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Set Up Weights & Biases for Experiment Tracking

In [8]:
try:
    from kaggle_secrets import UserSecretsClient
    wandb_key = UserSecretsClient().get_secret("WANDB_API_KEY")
    
    # Login to Weights & Biases
    import wandb
    wandb.login(key=wandb_key)
    wandb.init(project="sarvam-bengali-wikidata", name="finetuning-bengali-wikidata")
    use_wandb = True
except Exception as e:
    print(f"Error setting up wandb: {e}")
    print("Will continue without wandb tracking")
    use_wandb = False

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kitswas (kitswas-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250430_094655-1jketewn
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run finetuning-bengali-wikidata
wandb: ⭐️ View project at https://wandb.ai/kitswas-/sarvam-bengali-wikidata
wandb: 🚀 View run at https://wandb.ai/kitswas-/sarvam-bengali-wikidata/runs/1jketewn


## Train the Model

In [9]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

max_seq_length = 2048

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        report_to = "wandb",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        # # These enable multi-GPU training
        # local_rank=-1,  # Will be set by the launcher
        # ddp_backend="nccl"  # Best for GPUs
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
# Start training
try:
    trainer.train()
    training_successful = True
except Exception as e:
    print(f"Error during training: {e}")
    training_successful = False

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 186
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 100,368,384/1,506,920,448 (6.66% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.742000
2,2.651600
3,2.656000
4,2.749500
5,2.591900
6,2.677500
7,2.684900
8,2.641800
9,2.707300
10,2.473500


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


## Save Model and Finish Tracking

In [11]:
# Save the model
if training_successful:
    try:
        trainer.save_model("output-bengali-wikidata/final")
        print("Model saved successfully!")
    except Exception as e:
        print(f"Error saving model: {e}")

# Finish wandb tracking if it was used
if use_wandb:
    wandb.finish()

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Model saved successfully!


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         train/epoch ▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
wandb:   train/global_step ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
wandb:     train/grad_norm █▄▆▂▃▃▂▁▂▂▂▁▂▂▂▃▃▂▁▂▂▁▂▂▂▂▁▃▂▁▂▂▂▂▂▂▁▁▂▂
wandb: train/learning_rate ▁▅▅▇██▇▇▇▇▇▇▇▇▆▆▆▆▆▅▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁
wandb:          train/loss █▇▇██▇▆▄▆▆▅▅▄▅▄▆▆▅▄▅▃▅▅▅▃▄▄▆▄▅▅▅▅▁▅▅▅▅▂▆
wandb: 
wandb: Run summary:
wandb:               total_flos 4.143841841494426e+16
wandb:              train/epoch 2.96
wandb:        train/global_step 186
wandb:          train/grad_norm 0.12936
wandb:      train/learning_rate 0.0
wandb:               train/loss 2.4033
wandb:               train_loss 2.26479
wandb:            train_runtime 3214.911
wandb: train_samples_per_second 0.933
wandb:   train_steps_per_second 0.058
wandb: 
wandb: 🚀 View run finetuning-bengali-wikidata at: https://wandb.ai/kitswas-/sarvam-bengali-wikidata/runs/1jket

## Upload the Model to Hugging Face Hub

In [ ]:
import os

# Upload the model to Hugging Face Hub only if training was successful
if training_successful and os.path.exists("output-bengali-wikidata/final"):
    try:
        # Get HF token
        try:
            from kaggle_secrets import UserSecretsClient
            hf_token = UserSecretsClient().get_secret("HF_TOKEN")
        except Exception as e:
            print(f"Error getting HF token: {e}")
            print("Please enter your Hugging Face token manually:")
            hf_token = input()
        
        # Login to Hugging Face
        from huggingface_hub import login
        login(token=hf_token)
        
        from huggingface_hub import HfApi
        api = HfApi()
        
        # Create model card with dataset information
        model_card = """
        ---
        language: bn
        license: cc-by-nc-4.0
        datasets:
        - saurabhshahane/bangla-wikipedia
        ---
        
        # Sarvam-1 Fine-tuned on Bengali wikidata
        
        This model is a fine-tuned version of [sarvamai/sarvam-1](https://huggingface.co/sarvamai/sarvam-1)
        on the [Bengali wikidata dataset](https://www.kaggle.com/datasets/saurabhshahane/bangla-wikipedia).
        
        """
        
        with open("model_card.md", "w") as f:
            f.write(model_card)
        
        # Upload the model and model card
        api.upload_folder(
            folder_path="output-bengali-wikidata/final",
            path_in_repo=".",
            repo_id="kitswas/sarvam-1-finetuned-bengali-wikidata",
            repo_type="model",
            commit_message="Upload fine-tuned Sarvam-1 model on Bengali wikidata dataset",
        )
        
        api.upload_file(
            path_or_fileobj="model_card.md",
            path_in_repo="README.md",
            repo_id="kitswas/sarvam-1-finetuned-bengali-wikidata",
            repo_type="model",
            commit_message="Add model card",
        )
        
        print("Model uploaded successfully to the Hugging Face Hub!")
    except Exception as e:
        print(f"Error uploading model to Hugging Face Hub: {e}")
else:
    print("Model upload skipped - training was not successful or output directory does not exist.")


NameError: name 'os' is not defined